In [ ]:
# -*- coding=utf-8 -*-
"""
주요 기능: GoogleAuth  Google 인증 토큰 획득
    - GCP(Google Cloud Platform)에서 생성한 id/account/key 사용
        - user_nick(OAuth 2.0 Client ID)
        - bot_nick(Service Account)
        - api_key(API Key)  ## 구현되어 있지 않음
    - 현재 계정의 auth 사용
        - colab_auth: from colab import auth  auth.authenticate_user()

사용 방법: 
    - user_nick: gauth = GoogleAuth(user_nick="satsbymoon")  gd = GoogleDrive(gauth)
    - bot_nick: gauth = GoogleAuth(bot_nick="mats")  gd = GoogleDrive(gauth)

참조 자료:
    - 

필요 환경:
    - json 파일 생성 및 저장
        - user_nick
        - bot_nick
        - api_key
        - colab_auth
    - pip requirements:

"""

##@@@ Package/Module
##============================================================

##@@ Built-In Package/Module
##------------------------------------------------------------
import os, sys
import json
from pprint import pprint

## cURL 용
import subprocess
import io
from urllib.parse import urlparse, urlencode

##@@ External Package/Module
##------------------------------------------------------------

from googleapiclient.discovery import build
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
# from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

from oauth2client.service_account import ServiceAccountCredentials  ## bot_nick



##@@ User Package/Module
##------------------------------------------------------------

##@@@ Constant/Varible
##============================================================

## Local(PC Drive) 환경인지 Cloud(Google Drive) 환경인지 확인
ENV = "_LOCAL_"
if "content" in os.getcwd():
    ENV = "_GDRIVE_"

##@@ Constant
##------------------------------------------------------------

PATH_CONFIGS = "./configs"  if ENV == "_LOCAL_" else "/content/drive/MyDrive/__COLAB/CONFIGS/GOOGLE/"

##@@ Variable(Golobal)
##------------------------------------------------------------


##@@@ Private Function
##============================================================

##@@ path, credentials
##------------------------------------------------------------
def _get_path(path_type, nick="", settings_folder=PATH_CONFIGS):
    """

    """
    path = settings_folder[:-1] if settings_folder[-1] == "/" else settings_folder
    if path_type == 'token': # access token
        path += f"/{nick}_cred_token.json"
    elif path_type == 'user':  # client credentials
        path += f"/{nick}_cred_user.json"
    elif path_type == 'bot':  # client credentials
        path += f"/{nick}_cred_bot.json"
    elif path_type == 'spec':  # api spec(version, scope)
        path += f"/{nick}_api_spec.json"

    return path


def _set_credentials_by_user_from_sheets(user_nick):
    # TODO: 
    pass


def _set_credentials_by_user(nick):
    path_token = _get_path("token", nick=nick)
    path_user = _get_path("user", nick=nick)
    # api_spec = _get_path("spec", nick=nick)
    api_spec = json.load(open(_get_path("spec", nick=nick), "r", encoding="utf-8"))
    scopes = api_spec['scopes']
    creds = Credentials.from_authorized_user_file(path_token, scopes) if os.path.exists(path_token) else None

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # flow = InstalledAppFlow.from_client_secrets_file(PATH_CLIENT_SECRET.format(nick=nick), scopes)
            # https://google-auth-oauthlib.readthedocs.io/en/latest/reference/google_auth_oauthlib.flow.html
            # from_client_config(client_config, scopes, **kwargs)
            # from_client_secrets_file(client_secrets_file, scopes, **kwargs)
            flow = InstalledAppFlow.from_client_secrets_file(path_user, scopes)
            creds = flow.run_local_server(port=0)
        with open(path_token, 'w') as token:
            json.dump(json.loads(creds.to_json()), token, ensure_ascii=False, indent=4)
    
    # creds_info = {'creds': creds, 'version': api_spec['version']}
    # # creds_info = json.loads(creds.to_json())
    # creds_info['version'] = api_spec['version']
    return {'credentials': creds, 'version': api_spec['version']}


def _set_credentials_by_bot(nick):
    path_bot = _get_path("bot", nick=nick)
    api_spec = _get_path("spec", nick=nick)
    api_spec = json.load(open(_get_path("spec", nick=nick), "r", encoding="utf-8"))
    scopes = api_spec['scopes']

    credentials = ServiceAccountCredentials.from_json_keyfile_dict(
        json.load(open(path_bot, "r", encoding="utf-8")), 
        api_spec['scopes']
    )

    return {'credentials': credentials, 'version': api_spec['version']}


def _create_credentials(user_nick=None, bot_nick=None):
    if user_nick:
        return _set_credentials_by_user(nick=user_nick)
    elif bot_nick:
        return _set_credentials_by_bot(nick=bot_nick)
    else:
        return None


# ##@@ service
# ##------------------------------------------------------------
def _create_service_by_user_nick(api_name="sheets", nick=None, creds=None):
    try:
        service = build(api_name, creds['version'][api_name], credentials=creds['credentials'])
        print(f'{api_name} service(user_nick: {nick}) created successfully')
        return service
    except Exception as e:
        print(f'Unable to connect\n{e}')
        return None


def _create_service_by_bot_nick(api_name="sheets", nick=None, creds=None):
    try:
        service = build(api_name, creds['version'][api_name], credentials=creds['credentials'])
        print(f'{api_name} service(bot_nick: {nick}) created successfully')
        return service
    except Exception as e:
        print(f'Unable to connect\n{e}')
        return None


def _create_service(api_name, user_nick=None, bot_nick=None, creds=None):

    if user_nick:
        return _create_service_by_user_nick(api_name, nick=user_nick, creds=creds)
    elif bot_nick:
        return _create_service_by_bot_nick(api_name, nick=bot_nick, creds=creds)
    else:  ## colab auth
        pass


##@@@ Main Class
##============================================================
class GoogleAuth():
    def __init__(self, user_nick=None, bot_nick=None):
        creds_info = _create_credentials(user_nick=user_nick, bot_nick=bot_nick)
        self.credentials = creds_info['credentials']  # credentials
        self.version = creds_info['version']
        creds = json.loads(self.credentials.to_json())
        self.token = creds.get('token', None)
        
        # self.scopes = json.loads(self.credentials['credentials'].to_json())['scopes']
        # self.service = _create_service(api_name, user_nick=user_nick, bot_nick=bot_nick, creds=self.creds)
        # self.mime_type = _set_map_mime_type()


##@@@ Main Function
##============================================================
if __name__ == "__main__":
    pass
    ## path, credentials
    # user_nick = "moonitdev"
    user_nick = "satsbymoon"
    # # user_nick = "master"
    # bot_nick = "moonsats"

    ## NOTE: GoogleAuth
    gauth = GoogleAuth(user_nick=user_nick)
    # gauth = GoogleAuth(bot_nick=bot_nick)
    gds = build("drive", gauth.version["drive"], credentials = gauth.credentials)
    r = gds.files().list().execute()

    ## meta_data
    # r = gds.files().get(fileId="11XdqELYDyDea_4YLmJHSjSrimazLdtr-").execute()
    # 1ywx8F3Ujx0wH093wznXf0qgYo1mhKDRs
    # r = gds.files().get(fileId="1lJM_TdveFaVpieRBxvbDhYp6dQoKhFhc").execute()
    # r = gds.files().list(q="parents = '1ywx8F3Ujx0wH093wznXf0qgYo1mhKDRs' and trashed=false").execute()
    # file_id = "1m1iHuKPn_rQji2ByDyOHQZTAS6KQUTSW"
    # r = gds.files().copy(fileId="1m1iHuKPn_rQji2ByDyOHQZTAS6KQUTSW", body=dict(name="__GoogleREST.ipnb", mimeType="application/vnd.google.colaboratory")).execute()
    pprint(r)


    # # ##@@ NOTE: drive
    # gds = GoogleAPI("drive", user_nick=user_nick, bot_nick=None)
    # l = gds.token
    # # l = gds.list(queries=["parents = '1cXwU3nf9_CldBFWzb8DzJqlgs3sDyc7P'"], out_type="list")
    # # l = gss.id_by_name(name="Master_Configs_Google", app="sheets")
    # print(l)
    
    # gss = Google("sheets", user_nick=None, bot_nick=bot_nick)


